In [8]:
from dotenv import load_dotenv
import os

In [9]:
load_dotenv()

GOOGLE_API_KEY = os.environ['API_KEY']
os.environ['HF_HOME'] = 'D:\\Projects'

__DOCUMENT LOADING__

In [10]:
from langchain_community.document_loaders import PyPDFLoader

In [11]:
file_path = 'document/pdf_document.pdf'
loader = PyPDFLoader(file_path)

pages = []
for doc in loader.lazy_load():
    pages.append(doc)

In [12]:
len(pages)
# pages[127].page_content[:100]

258

__CHUNK SPLITTING__

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [14]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=500,
    length_function=len,
    is_separator_regex=False,
)

In [15]:
splits = splitter.split_documents(pages)

In [16]:
len(splits)

287

__EMBEDDING__

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings

In [18]:
embeddings = HuggingFaceEmbeddings(
    model="BAAI/bge-small-en-v1.5"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

D:\Projects\RAG_implementation\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\Projects\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
from langchain_chroma import Chroma

In [20]:
vector_store = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory='./chroma-vectordb',
)

In [22]:
vector_store._collection.count()

287